import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [5]:
import findspark
findspark.init()

In [6]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

create local SparkSession

In [7]:
spark = SparkSession.builder.getOrCreate()

In [8]:
spark

read csv with inferschema

In [9]:
df = spark.read.csv(
   "ds_salaries.csv",
   header = True,
   inferSchema = True
    )

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [10]:
spark.read.format("csv").options(header = True,inferSchema = True ).load("ds_salaries.csv")

DataFrame[id: int, work_year: int, experience_level: string, employment_type: string, job_title: string, salary: int, salary_currency: string, salary_in_usd: int, employee_residence: string, remote_ratio: int, company_location: string, company_size: string]

write schema of scv on screen

In [11]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [12]:
schema =  StructType([\
    StructField('id', IntegerType(),False), \
    StructField('work_year', IntegerType(),False), \
    StructField('experience_level', StringType(),False), \
    StructField('employment_type', StringType(),False), \
    StructField('job_title', StringType(),False), \
    StructField('salary', FloatType(),False), \
    StructField('salary_currency', StringType(),False), \
    StructField('salary_in_usd', FloatType(),False), \
    StructField('employee_residence', StringType(),False), \
    StructField('remote_ratio', IntegerType(),False), \
    StructField('company_location', StringType(),False), \
    StructField('company_size', StringType(),False) \
])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [13]:
df = spark.read.csv(
   "ds_salaries.csv",
   header = True,
   schema = schema
    )

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [14]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: float (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: float (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [15]:
df.show()

+---+---------+----------------+---------------+--------------------+---------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experience_level|employment_type|           job_title|   salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+---------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|  70000.0|            EUR|      79833.0|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...| 260000.0|            USD|     260000.0|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|  85000.0|            GBP|     109024.0|                GB|          50

print data in dataframe using display(df.toPandas())

In [16]:
df.toPandas()

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000.0,EUR,79833.0,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000.0,USD,260000.0,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000.0,GBP,109024.0,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000.0,USD,20000.0,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000.0,USD,150000.0,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000.0,USD,154000.0,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000.0,USD,126000.0,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000.0,USD,129000.0,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000.0,USD,150000.0,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [17]:
df_job_title = df.select(col("job_title")).distinct()

print all rows from df_job_titles without truncating jobs

In [18]:
df_job_title.show(df_job_title.count(),truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
|Research S

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [19]:
df_analytic = df.groupby("job_title").agg(
    avg("salary_in_usd").alias("avg_salary"),
    min("salary_in_usd").alias("min_salary"),
    max("salary_in_usd").alias("max_salary")
)

print all rows from df_analytic without trancating jobs

In [20]:
df_analytic.show(df_analytic.count(),truncate=False)

+----------------------------------------+------------------+----------+----------+
|job_title                               |avg_salary        |min_salary|max_salary|
+----------------------------------------+------------------+----------+----------+
|3D Computer Vision Researcher           |5409.0            |5409.0    |5409.0    |
|Lead Data Engineer                      |139724.5          |56000.0   |276000.0  |
|Head of Machine Learning                |79039.0           |79039.0   |79039.0   |
|Data Specialist                         |165000.0          |165000.0  |165000.0  |
|Data Analytics Lead                     |405000.0          |405000.0  |405000.0  |
|Machine Learning Scientist              |158412.5          |12000.0   |260000.0  |
|Lead Data Analyst                       |92203.0           |19609.0   |170000.0  |
|Data Engineering Manager                |123227.2          |59303.0   |174000.0  |
|Staff Data Scientist                    |105000.0          |105000.0  |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [21]:
window_row_number = Window.orderBy(col("avg_salary").desc())

df_analytic = df_analytic.withColumn("row_id", row_number().over(window_row_number))

print all data from df_analytic

In [22]:
df_analytic.show(df_analytic.count(),truncate=False)

24/05/15 10:50:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/15 10:50:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/15 10:50:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/15 10:50:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/15 10:50:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/15 10:50:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------------------------------+------------------+----------+----------+------+
|job_title                               |avg_salary        |min_salary|max_salary|row_id|
+----------------------------------------+------------------+----------+----------+------+
|Data Analytics Lead                     |405000.0          |405000.0  |405000.0  |1     |
|Principal Data Engineer                 |328333.3333333333 |185000.0  |600000.0  |2     |
|Financial Data Analyst                  |275000.0          |100000.0  |450000.0  |3     |
|Principal Data Scientist                |215242.42857142858|148261.0  |416000.0  |4     |
|Director of Data Science                |195074.0          |130026.0  |325000.0  |5     |
|Data Architect                          |177873.9090909091 |90700.0   |266400.0  |6     |
|Applied Data Scientist                  |175655.0          |54238.0   |380000.0  |7     |
|Analytics Engineer                      |175000.0          |135000.0  |205300.0  |8     |

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [23]:
df_analytic = df_analytic.select("row_id", "job_title", "avg_salary", "min_salary", "max_salary")

print df_analytic now

In [24]:
df_analytic.show(df_analytic.count(),truncate=False)

+------+----------------------------------------+------------------+----------+----------+
|row_id|job_title                               |avg_salary        |min_salary|max_salary|
+------+----------------------------------------+------------------+----------+----------+
|1     |Data Analytics Lead                     |405000.0          |405000.0  |405000.0  |
|2     |Principal Data Engineer                 |328333.3333333333 |185000.0  |600000.0  |
|3     |Financial Data Analyst                  |275000.0          |100000.0  |450000.0  |
|4     |Principal Data Scientist                |215242.42857142858|148261.0  |416000.0  |
|5     |Director of Data Science                |195074.0          |130026.0  |325000.0  |
|6     |Data Architect                          |177873.9090909091 |90700.0   |266400.0  |
|7     |Applied Data Scientist                  |175655.0          |54238.0   |380000.0  |
|8     |Analytics Engineer                      |175000.0          |135000.0  |205300.0  |

24/05/15 10:50:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/15 10:50:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/15 10:50:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/15 10:50:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/15 10:50:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/15 10:50:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [25]:
df_exp_lvl = df.groupBy("experience_level").agg(max("salary_in_usd").alias("biggest_salary"))

print here df_exp_lvl

In [26]:
df_exp_lvl.show(df_exp_lvl.count(),truncate=False)

+----------------+--------------+
|experience_level|biggest_salary|
+----------------+--------------+
|EX              |600000.0      |
|MI              |450000.0      |
|EN              |250000.0      |
|SE              |412000.0      |
+----------------+--------------+



create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [27]:
df_best = df.select("id", "experience_level", "employee_residence", col("salary_in_usd").alias("biggest_salary")).join(df_exp_lvl, ["experience_level","biggest_salary"],"inner")
df_best = df_best.select("id", "experience_level", "employee_residence","biggest_salary")

print df_best

In [28]:
df_best.show(df_best.count(),truncate=False)

+---+----------------+------------------+--------------+
|id |experience_level|employee_residence|biggest_salary|
+---+----------------+------------------+--------------+
|33 |MI              |US                |450000.0      |
|37 |EN              |US                |250000.0      |
|63 |SE              |US                |412000.0      |
|97 |MI              |US                |450000.0      |
|252|EX              |US                |600000.0      |
+---+----------------+------------------+--------------+



drop duplicates if exist by experience_level

In [29]:
df_best = df_best.dropDuplicates(["experience_level"])

print df_best

In [30]:
df_best.show(df_best.count(),truncate=False)

+---+----------------+------------------+--------------+
|id |experience_level|employee_residence|biggest_salary|
+---+----------------+------------------+--------------+
|37 |EN              |US                |250000.0      |
|252|EX              |US                |600000.0      |
|33 |MI              |US                |450000.0      |
|63 |SE              |US                |412000.0      |
+---+----------------+------------------+--------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [31]:
df_new_best = df_best.withColumn("experience_level", when(col("experience_level") == "MI","middle")
                                                                               .when(col("experience_level") == "SE","senior")
                                                                               .otherwise("Null")
                                                                               )
df_new_best = df_new_best.select("experience_level","employee_residence", "biggest_salary")

print df_new_best

In [32]:
df_new_best.show(df_new_best.count(),truncate=False)

+----------------+------------------+--------------+
|experience_level|employee_residence|biggest_salary|
+----------------+------------------+--------------+
|Null            |US                |250000.0      |
|Null            |US                |600000.0      |
|middle          |US                |450000.0      |
|senior          |US                |412000.0      |
+----------------+------------------+--------------+



write df_new_best like 1.csv and load then it to df_final

In [40]:
df_new_best.write.options(header = True, delimiter=',').format("csv").save("./df_new_best_csv")

#df_new_best.toPandas().to_csv("./df_new_best.csv")


In [47]:
df_final =  spark.read.csv(
   "df_new_best_csv/part-00000-3ec71c9e-afe2-4cb7-9d87-7d4aa06e9acb-c000.csv",
   header = True,
   inferSchema = True
    )

print df_final

In [48]:
df_final.show(df_final.count(),truncate=False)

+----------------+------------------+--------------+
|experience_level|employee_residence|biggest_salary|
+----------------+------------------+--------------+
|Null            |US                |250000.0      |
|Null            |US                |600000.0      |
|middle          |US                |450000.0      |
|senior          |US                |412000.0      |
+----------------+------------------+--------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [35]:
df_final = df_final.select("experience_level", "employee_residence", col("biggest_salary").alias("salary_in_usd")) \
.filter(col("experience_level") != "Null").join(df, ["salary_in_usd","employee_residence"], "inner")

print df_final

In [36]:
df_final.show(df_final.count(),truncate=False)

+-------------+------------------+----------------+---+---------+----------------+---------------+----------------------+--------+---------------+------------+----------------+------------+
|salary_in_usd|employee_residence|experience_level|id |work_year|experience_level|employment_type|job_title             |salary  |salary_currency|remote_ratio|company_location|company_size|
+-------------+------------------+----------------+---+---------+----------------+---------------+----------------------+--------+---------------+------------+----------------+------------+
|450000.0     |US                |middle          |33 |2020     |MI              |FT             |Research Scientist    |450000.0|USD            |0           |US              |M           |
|412000.0     |US                |senior          |63 |2020     |SE              |FT             |Data Scientist        |412000.0|USD            |100         |US              |L           |
|450000.0     |US                |middle          

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [37]:
max_salary_in_usd = df_final.select(max("salary_in_usd")).collect()[0][0]
print(max_salary_in_usd)

450000.0


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system